In [22]:
import lazypredict
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

#Read data file

import pandas as pd

filepath = "balanced_dataset2.csv"
df = pd.read_csv(filepath)
features = df

features

,page_id,page_title,view_count,protection_level,protection_expiry,page_length,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories
0,58751408,Legion,724996,sysop,infinity,54,25,25,0,0,2,0,0,2
1,27438041,Battle For The Sun,7194,sysop,infinity,64,25,25,0,22,2,0,0,2
2,31140892,Made for Each Other (album),7006,unprotected,NaN,69,25,25,0,0,2,0,0,2
3,46755229,Foregen,5809,sysop,infinity,76,25,25,0,79,2,0,0,2
4,58751406,LEGION,5081,sysop,infinity,54,25,25,0,0,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,8561806,Princha Khen,1152,unprotected,NaN,0,25,25,0,0,962,4,2,89
4763,48988639,Juliana Castro (volleyball),959,unprotected,NaN,0,25,25,0,0,962,4,2,89
4764,0,Amala ayurvedic hospital & research centre,618,sysop,infinity,0,25,25,0,0,962,4,2,89
4765,50225375,Dandamudi Sumathi,247,unprotected,NaN,0,25,25,0,0,962,4,2,89


In [23]:
# Labels are the values we want to predict
labels = np.array(df['protection_level'])

labels

array(['sysop', 'sysop', 'unprotected', 'sysop', 'sysop', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'unprotected', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'unprotected', 'unprotected', 'sysop', 'autoconfirmed', 'sysop',
       'sysop', 'sysop', 'unprotected', 'sysop', 'autoconfirmed', 'sysop',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'unprotected',
       'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'unprotected', 'unprotected', 'sysop', 'autoconfirmed', 'sysop',
       'unprotected', 'sysop', 'sysop', 'sysop', 'sysop', 'sysop',
       'sysop', 'unprotected', 'sysop', 'sysop', 'autoconfirmed',
       'autoconfirmed', 'unprotected

In [24]:
# 0 => unprotected
# 1 => autoconfirmed
# 2 => extendedconfirmed
# 3 => sysop
labels_encoded = []
for item in labels:
    if(item =="unprotected"):
        labels_encoded.append(0)
    elif(item == "autoconfirmed"):
        labels_encoded.append(1)
    elif(item == "extendedconfirmed"):
        labels_encoded.append(2)
    elif(item == "sysop"):
        labels_encoded.append(3)  
labels_encoded

[3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 3,
 1,
 3,
 3,
 3,
 0,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 3,
 1,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 1,
 1,
 0,
 3,
 3,
 0,
 3,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 1,
 3,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 2,
 0,
 3,
 0,
 3,
 0,
 3,
 0,
 0,
 3,
 3,
 2,
 0,
 0,
 2,
 2,
 3,
 0,
 3,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 0,
 1,
 3,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 2,
 3,
 3,
 3,
 1,
 3,
 2,
 1,
 3,
 1,
 0,
 2,
 2,
 1,
 3,
 2,
 1,
 0,
 0,
 0,
 1,
 3,
 2,
 2,
 1,
 3,
 2,
 0,
 1,
 3,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 3,
 3,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,


In [25]:
# Remove the labels from the features
features = features.drop('protection_level', axis = 1)
features = features.drop('page_title', axis = 1)
features = features.drop('protection_expiry', axis = 1)
features = features.drop('page_id', axis = 1)
features

,view_count,page_length,number_page_watchers,number_page_watchers_recent_edits,number_of_redirects,page_views_past_30days,total_edits,recent_number_of_edits,number_distinct_authors,number_categories
0,724996,54,25,25,0,0,2,0,0,2
1,7194,64,25,25,0,22,2,0,0,2
2,7006,69,25,25,0,0,2,0,0,2
3,5809,76,25,25,0,79,2,0,0,2
4,5081,54,25,25,0,0,2,0,0,2
...,...,...,...,...,...,...,...,...,...,...
4762,1152,0,25,25,0,0,962,4,2,89
4763,959,0,25,25,0,0,962,4,2,89
4764,618,0,25,25,0,0,962,4,2,89
4765,247,0,25,25,0,0,962,4,2,89


In [26]:
features['page_length'] = features['page_length'].astype(float)
features['edit_count'] = features['total_edits'].astype(float)
features['page_watchers'] = features['number_page_watchers'].astype(float)
features['page_watchers_recent_edits'] = features['number_page_watchers_recent_edits'].astype(float)

In [27]:
# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

features

array([[7.24996000e+05, 5.40000000e+01, 2.50000000e+01, 2.50000000e+01,
        0.00000000e+00, 0.00000000e+00, 2.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.50000000e+01,
        2.50000000e+01],
       [7.19400000e+03, 6.40000000e+01, 2.50000000e+01, 2.50000000e+01,
        0.00000000e+00, 2.20000000e+01, 2.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.50000000e+01,
        2.50000000e+01],
       [7.00600000e+03, 6.90000000e+01, 2.50000000e+01, 2.50000000e+01,
        0.00000000e+00, 0.00000000e+00, 2.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.50000000e+01,
        2.50000000e+01],
       [5.80900000e+03, 7.60000000e+01, 2.50000000e+01, 2.50000000e+01,
        0.00000000e+00, 7.90000000e+01, 2.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 2.00000000e+00, 2.50000000e+01,
        2.50000000e+01],
       [5.08100000e+03, 5.40000000e+

In [28]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size =0.20, random_state = 53)

X_train = train_features
y_train = train_labels
X_test = test_features
y_test = test_labels


In [29]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:11<00:00,  2.56it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
RandomForestClassifier             0.75               0.75    None      0.74   
LGBMClassifier                     0.75               0.75    None      0.74   
XGBClassifier                      0.74               0.75    None      0.74   
ExtraTreesClassifier               0.74               0.74    None      0.73   
BaggingClassifier                  0.73               0.73    None      0.72   
KNeighborsClassifier               0.70               0.70    None      0.69   
AdaBoostClassifier                 0.67               0.68    None      0.66   
DecisionTreeClassifier             0.67               0.67    None      0.67   
ExtraTreeClassifier                0.63               0.64    None      0.63   
LogisticRegression                 0.61               0.62    None      0.61   
SVC                                0.61 